In [10]:
import open3d as o3d
import cl
import utils as ut
import numpy as np
from skimage.morphology import binary_dilation
import proc3d
import json
from PIL import Image
from utils import *
import glob
import os
%matplotlib widget 
#%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import glob

In [7]:
class space_carving():
    def __init__(self, dataset_path):
        self.images = self.load_images(os.path.join(dataset_path, 'imgs'))
        self.extrinsics = self.load_extrinsics(os.path.join(dataset_path, 'extrinsics'))
        #self.bbox = json.load(open(os.path.join(dataset_path, 'bbox.json')))
        self.bbox = json.load(open(os.path.join(dataset_path, '/home/pico/uni/romi/scanner_cube/bbox_min_max.json')))
        
        self.camera_model = json.load(open(os.path.join(dataset_path, 'camera_model.json')))
        self.intrinsics= self.camera_model['params'][0:4]
        
        params = json.load(open(os.path.join(dataset_path, 'params.json')))
        self.gt=o3d.io.read_point_cloud(params["gt_path"])
        self.gt_points = np.asarray(self.gt.points)
        self.n_dilation=params["sc"]["n_dilation"]
        self.voxel_size = params['sc']['voxel_size']
        
        self.set_sc(self.bbox)
        
    def reset(self):
        del(self.sc)
        self.set_sc(self.bbox) 
        
    def load_extrinsics(self,path):
        ext = []
        ext_files = glob.glob(os.path.join(path, '*.json'))
        assert len(ext_files) != 0,"json list is empty."
        for i in sorted(ext_files):                                                                                                                                     
            ext.append(json.load(open(i)))                                                                                                                                                                                                                                                  
        return ext 
    
    def load_images(self,path):                                                                                                                                         
        imgs = []
        img_files = glob.glob(os.path.join(path, '*.png')) #get all .png files from folder path
        assert len(img_files) != 0,"Image list is empty."
        for i in sorted(img_files):                                                                                                                                     
            img = Image.open(i)                                                                                                                                      
            imgs.append(img.copy())                                                                                                                                     
            img.close()                                                                                                                                                                                                                                                   
        return imgs

    def set_sc(self,bbox):
        x_min, x_max = bbox['x']
        y_min, y_max = bbox['y']
        z_min, z_max = bbox['z']

        nx = int((x_max - x_min) / self.voxel_size) + 1
        ny = int((y_max - y_min) / self.voxel_size) + 1
        nz = int((z_max - z_min) / self.voxel_size) + 1
        
        print(nx,ny,nz)

        self.origin = np.array([x_min, y_min, z_min])
        self.sc = cl.Backprojection([nx, ny, nz], [x_min, y_min, z_min], self.voxel_size)

    def carve(self,idx):
        self.space_carve(self.images[idx], self.extrinsics[idx])
        
    def space_carve(self, im, rt):
        mask = get_mask(im)
        rot = sum(rt['R'], [])
        tvec = rt['T']
        if self.n_dilation:
            for k in range(self.n_dilation): mask = binary_dilation(mask)    
        self.sc.process_view(self.intrinsics, rot, tvec, mask)
        
    def dist_to_gt(self):
        vol = self.sc.values().copy()
        vol = vol.reshape(self.sc.shape)
        pcd=proc3d.vol2pcd_exp(vol, self.origin, self.voxel_size, level_set_value=0) 
        pcd_p = np.asarray(pcd.points)
        cd=chamfer_d(self.gt_points , pcd_p)
        return cd

In [3]:
class space_carving_2():
    def __init__(self, dataset_path):
        self.img_files = sorted (glob.glob(os.path.join(dataset_path, 'imgs', '*.png')) )#get all .png file names from folder path
        self.extrinsics = self.load_extrinsics(os.path.join(dataset_path, 'extrinsics'))
        #self.bbox = json.load(open(os.path.join(dataset_path, 'bbox.json')))
        self.bbox = json.load(open(os.path.join(dataset_path, '/home/pico/uni/romi/scanner_cube/bbox_min_max.json')))
        self.camera_model = json.load(open(os.path.join(dataset_path, 'camera_model.json')))
        self.intrinsics= self.camera_model['params'][0:4]
        
        params = json.load(open(os.path.join(dataset_path, 'params.json')))
        self.gt=o3d.io.read_point_cloud(params["gt_path"])
        self.gt_points = np.asarray(self.gt.points)
        self.n_dilation=params["sc"]["n_dilation"]
        self.voxel_size = params['sc']['voxel_size']
        
        self.set_sc(self.bbox)
        
    def reset(self):
        del(self.sc)
        self.set_sc(self.bbox) 
        
    def load_extrinsics(self,path):
        ext = []
        ext_files = glob.glob(os.path.join(path, '*.json'))
        assert len(ext_files) != 0,"json list is empty."
        for i in sorted(ext_files):                                                                                                                                     
            ext.append(json.load(open(i)))                                                                                                                                                                                                                                                  
        return ext 
    
    def load_image(self,idx):                                                                                                                                         
        img = Image.open(self.img_files[idx])                                                                                                                                      
        cp = img.copy()                                                                                                                                 
        img.close()                                                                                                                                                                                                                                                   
        return cp

    def set_sc(self,bbox):
        x_min, x_max = bbox['x']
        y_min, y_max = bbox['y']
        z_min, z_max = bbox['z']

        nx = int((x_max - x_min) / self.voxel_size) + 1
        ny = int((y_max - y_min) / self.voxel_size) + 1
        nz = int((z_max - z_min) / self.voxel_size) + 1

        self.origin = np.array([x_min, y_min, z_min])
        self.sc = cl.Backprojection([nx, ny, nz], [x_min, y_min, z_min], self.voxel_size)

    def carve(self,idx):
        im = self.load_image(idx)
        self.space_carve(im, self.extrinsics[idx])
        
    def space_carve(self, im, rt):
        mask = get_mask(im)
        rot = sum(rt['R'], [])
        tvec = rt['T']
        if self.n_dilation:
            for k in range(self.n_dilation): mask = binary_dilation(mask)    
        self.sc.process_view(self.intrinsics, rot, tvec, mask)
        
    def dist_to_gt(self):
        vol = self.sc.values().copy()
        vol = vol.reshape(self.sc.shape)
        pcd=proc3d.vol2pcd_exp(vol, self.origin, self.voxel_size, level_set_value=0) 
        pcd_p = np.asarray(pcd.points)
        cd=chamfer_d(self.gt_points , pcd_p)
        return cd

In [11]:
class space_carving_2_masks():
    def __init__(self, dataset_path,gt_mode=False):
        self.masks_files = sorted (glob.glob(os.path.join(dataset_path, 'masks', '*.png')) )#get all .png file names from folder path
        self.extrinsics = self.load_extrinsics(os.path.join(dataset_path, 'extrinsics'))
        #self.bbox = json.load(open(os.path.join(dataset_path, 'bbox.json')))
        self.bbox = json.load(open(os.path.join(dataset_path, '/home/pico/uni/romi/scanner_cube/bbox_min_max.json')))
        self.camera_model = json.load(open(os.path.join(dataset_path, 'camera_model.json')))
        self.intrinsics= self.camera_model['params'][0:4]
        
        params = json.load(open(os.path.join(dataset_path, 'params.json')))


        #self.gt=o3d.io.read_point_cloud(params["gt_path"])
        #self.gt_points = np.asarray(self.gt.points)
        self.gt_mode = gt_mode

        if self.gt_mode is True:
            self.gt = np.load(os.path.join(dataset_path, 'volumes','vol_180.npy'))
            self.gt_solid_mask = np.where(self.gt==1,True,False) 
            self.gt_n_solid_voxels = np.count_nonzero(self.gt_solid_mask)


        self.n_dilation=params["sc"]["n_dilation"]
        self.voxel_size = params['sc']['voxel_size']
        
        self.set_sc(self.bbox)
        
    def reset(self):
        del(self.sc)
        self.set_sc(self.bbox) 
        
    def load_extrinsics(self,path):
        ext = []
        ext_files = glob.glob(os.path.join(path, '*.json'))
        assert len(ext_files) != 0,"json list is empty."
        for i in sorted(ext_files):                                                                                                                                     
            ext.append(json.load(open(i)))                                                                         
        return ext 
    
    def load_mask(self,idx):                                                                                                                                         
        img = cv2.imread(self.masks_files[idx], cv2.IMREAD_GRAYSCALE)
        return img

    def set_sc(self,bbox):
        x_min, x_max = bbox['x']
        y_min, y_max = bbox['y']
        z_min, z_max = bbox['z']

        nx = int((x_max - x_min) / self.voxel_size) + 1
        ny = int((y_max - y_min) / self.voxel_size) + 1
        nz = int((z_max - z_min) / self.voxel_size) + 1

        self.origin = np.array([x_min, y_min, z_min])
        self.sc = cl.Backprojection([nx, ny, nz], [x_min, y_min, z_min], self.voxel_size)

    def carve(self,idx):
        im = self.load_mask(idx)
        self.space_carve(im, self.extrinsics[idx])
        
    def space_carve(self, mask, rt):
        #mask = im.copy() #get_mask(im)
        rot = sum(rt['R'], [])
        tvec = rt['T']
        if self.n_dilation:
            for k in range(self.n_dilation): mask = binary_dilation(mask)    
        self.sc.process_view(self.intrinsics, rot, tvec, mask)


    def gt_compare(self,test_vol):
        if self.gt_mode is False:
            return 0
        #compare current volume with ground truth (voxelwise) and return percentage
        comp = np.where( self.gt==test_vol,True,False)
        eq_count = np.count_nonzero(comp)
        #perc_sim = (eq_count/np.prod(gt_vol.shape) )*100.
        #perc_sim = (eq_count/682176)*100. #682176number of voxels of the volumes used here 
        perc_sim = eq_count * 0.00014658973637301812
        return perc_sim
    
    def gt_compare_solid(self,test_vol):
        if self.gt_mode is False:
            return 0
        #compares only solid voxels (with 1;s) between ground truth and test_vol  
        vol_solid_mask = np.where(test_vol==1,True,False) 
        vol_n_solid_voxels = np.count_nonzero(vol_solid_mask)
        intersection = self.gt_solid_mask & vol_solid_mask
        n_intersection = np.count_nonzero(intersection)
        ratio = n_intersection / ( self.gt_n_solid_voxels + vol_n_solid_voxels - n_intersection )
        return ratio
    
    '''def dist_to_gt(self):
        vol = self.sc.values().copy()
        vol = vol.reshape(self.sc.shape)
        pcd=proc3d.vol2pcd_exp(vol, self.origin, self.voxel_size, level_set_value=0) 
        pcd_p = np.asarray(pcd.points)
        cd=chamfer_d(self.gt_points , pcd_p)
        return cd'''


In [18]:
#data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/000'
#spc = space_carving(data_path)

#for i in range(180):
#    spc.carve(i)

# for creating a responsive plot
%matplotlib widget
  
# importing required libraries
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
  
x,y,z = np.where(spc.sc.values()[:64,:64,:128]==1)
  
# creating figure
fig = plt.figure()
ax = Axes3D(fig)

ax.set_xlim3d(0, 64)
ax.set_ylim3d(0, 64)
ax.set_zlim3d(0, 128)

#ax.set_xlim3d(0, 140)
#ax.set_ylim3d(0, 140)
#ax.set_zlim3d(0, 140)
  
# creating the plot
plot_geeks = ax.scatter(x, y, z, color='green',s=1)

x,y,z = np.where(spc.sc.values()==0)
#plot_geeks = ax.scatter(x, y, z, color='red',s=1)
  
# setting title and labels
ax.set_title("3D plot")
ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('z-axis')
ax.grid()
  
# displaying the plot
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
np.histogram(spc.sc.values(), bins=3)

In [16]:
data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/204'#'
spc = space_carving_2_masks(data_path)

In [ ]:
spc.carve(45)
print(np.histogram(spc.sc.values(), bins=3)[0])

In [ ]:
print(np.histogram(spc.sc.values(), bins=3)[0])

In [ ]:
ant = 0
for i in range(0,180,40):
    spc.carve(i)
    h = np.histogram(spc.sc.values(), bins=3)
    print(i,h[0],h[0][0]-ant)
    ant=h[0][0]

In [17]:

pos = np.random.randint(180, size=10)
for i in range(180):#pos:
    spc.carve(i)
    print(i,np.histogram(spc.sc.values(), bins=3)[0])


0 [395109 177999 109068]
1 [401372 177347 103457]
2 [405121 176853 100202]
3 [408838 176336  97002]
4 [412852 175710  93614]
5 [417207 175063  89906]
6 [421805 174403  85968]
7 [426864 173684  81628]
8 [431724 172998  77454]
9 [435703 172204  74269]
10 [440022 171372  70782]
11 [442541 170608  69027]
12 [444887 169782  67507]
13 [447230 168870  66076]
14 [449857 168009  64310]
15 [451509 167090  63577]
16 [452835 166167  63174]
17 [454636 165190  62350]
18 [456058 164229  61889]
19 [457597 163273  61306]
20 [459011 162313  60852]
21 [460459 161278  60439]
22 [461766 160367  60043]
23 [463308 159275  59593]
24 [464943 158168  59065]
25 [466206 157308  58662]
26 [467635 156291  58250]
27 [468937 155276  57963]
28 [470059 154312  57805]
29 [471731 153309  57136]
30 [473058 152341  56777]
31 [474272 151349  56555]
32 [475812 150348  56016]
33 [477027 149384  55765]
34 [478291 148399  55486]
35 [479667 147400  55109]
36 [481071 146419  54686]
37 [482401 145446  54329]
38 [483709 144447  540

In [ ]:
spc.sc.values().shape

In [ ]:
#data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/'
data_path = '/home/pico/uni/romi/rl_sony/arabidopsis_image_sets'
gdeltas = []
for plantn in [202]:#range(180):
    spc = space_carving_2(os.path.join(data_path,str(plantn).zfill(3)))
    for i in range(180):
        deltas = []
        pos = np.random.randint(180, size=10)
        n_ant = 0
        for p in pos:
            spc.carve(p)
            espaces = np.histogram(spc.sc.values(), bins=3)[0][0]
            delta = espaces-n_ant
            deltas.append(delta)
            n_ant = espaces
        print(deltas)
        gdeltas.append(deltas)

In [21]:
data =[]
for bias in range(180):
    data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/204'
    spc = space_carving_2_masks(data_path)
    for i in range(0,180,10):
        spc.carve(calculate_position(i,bias))
    h = np.histogram(spc.sc.values(), bins=3)[0]
    data.append(h)
    print(bias,h)

print(np.mean(data,axis=0))

0 [626601  11215  44360]
1 [626418  11216  44542]
2 [628229  11210  42737]
3 [629678  11208  41290]
4 [628930  11219  42027]
5 [628003  11229  42944]
6 [629223  11225  41728]
7 [629897  11223  41056]
8 [629682  11218  41276]
9 [629757  11220  41199]
10 [626601  11215  44360]
11 [626418  11216  44542]
12 [628229  11210  42737]
13 [629678  11208  41290]
14 [628930  11219  42027]
15 [628003  11229  42944]
16 [629223  11225  41728]
17 [629897  11223  41056]
18 [629682  11218  41276]
19 [629757  11220  41199]
20 [626601  11215  44360]
21 [626418  11216  44542]
22 [628229  11210  42737]
23 [629678  11208  41290]
24 [628930  11219  42027]
25 [628003  11229  42944]
26 [629223  11225  41728]
27 [629897  11223  41056]
28 [629682  11218  41276]
29 [629757  11220  41199]
30 [626601  11215  44360]
31 [626418  11216  44542]
32 [628229  11210  42737]
33 [629678  11208  41290]
34 [628930  11219  42027]
35 [628003  11229  42944]
36 [629223  11225  41728]
37 [629897  11223  41056]
38 [629682  11218  412

In [ ]:
dt = np.array(gdeltas)

In [ ]:
dt.shape

In [22]:
data =[]
for n in range(180):
    data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/204'
    spc = space_carving_2_masks(data_path)
    pos = np.random.randint(180, size=18)
    for i in pos:
        spc.carve(i)
    h = np.histogram(spc.sc.values(), bins=3)[0]
    data.append(h)
    print(h)

print(np.mean(data,axis=0))

[617096  12367  52713]
[617250  11747  53179]
[627924  11369  42883]
[621568  11561  49047]
[632480  11414  38282]
[631326  11335  39515]
[628576  11470  42130]
[626794  11656  43726]
[629157  11692  41327]
[617627  12265  52284]
[603710  11870  66596]
[617024  12008  53144]
[624652  12398  45126]
[627197  11545  43434]
[622260  11399  48517]
[618488  12459  51229]
[623949  11468  46759]
[627823  11436  42917]
[622019  11487  48670]
[619089  11359  51728]
[630199  11497  40480]
[629490  11621  41065]
[619368  11579  51229]
[620102  12550  49524]
[626514  11481  44181]
[607229  13789  61158]
[625852  12392  43932]
[620274  11971  49931]
[622398  11448  48330]
[626854  12242  43080]
[622952  11556  47668]
[628466  11491  42219]
[624183  11524  46469]
[633119  11638  37419]
[606946  12029  63201]
[608881  11457  61838]
[619234  12226  50716]
[631637  11492  39047]
[622741  11633  47802]
[610021  11654  60501]
[620471  11643  50062]
[626376  11538  44262]
[600725  11817  69634]
[628203  11

In [ ]:
np.mean(dt,axis=0)

In [ ]:
np.min(dt,axis=0)

In [ ]:
np.max(dt,axis=0)

In [ ]:
np.std(dt,axis=0)

In [14]:
def calculate_position(init_state,steps):
        n_positions = 180
        n_pos = init_state + steps
        if n_pos>(n_positions-1):
            n_pos -= n_positions
        elif n_pos<0:
            n_pos += n_positions
        return n_pos

In [ ]:
rel_pos = 0
p_bias = 179
abs_pos = calculate_position(rel_pos,p_bias)
print(abs_pos)

In [ ]:
spc.sc.values().shape

In [5]:
spc.sc.values().shape

(66, 68, 152)

In [6]:
66*68*152

682176

In [23]:
def test_uniform(data_path,n_images):
    data =[]
    for bias in range(180):
        spc = space_carving_2_masks(data_path)
        dist = 180//n_images
        for i in range(0,180,dist):
            spc.carve(calculate_position(i,bias))
        h = np.histogram(spc.sc.values(), bins=3)[0]
        data.append(h)
        print(bias,h)
    return data

def test_random(data_path,n_images):
    data =[]
    for n in range(180):
        spc = space_carving_2_masks(data_path)
        pos = np.random.randint(180, size=n_images)
        for i in pos:
            spc.carve(i)
        h = np.histogram(spc.sc.values(), bins=3)[0]
        data.append(h)
        print(h)
    return data

def test_all(data_path):
    spc = space_carving_2_masks(data_path)
    for i in range(180):#pos:
        spc.carve(i)
        count = np.histogram(spc.sc.values(), bins=3)[0]
        print(i,count)
    return count

In [12]:
def compare_vols(gt_vol,test_vol):
    comp = np.where( gt_vol==test_vol,True,False)
    eq_count = np.count_nonzero(comp)
    #perc_sim = (eq_count/np.prod(gt_vol.shape) )*100.
    #perc_sim = (eq_count/682176)*100. #682176number of voxels of the volumes used here 
    perc_sim = eq_count * 0.00014658973637301812
    return perc_sim 

def test_uniform(data_path,n_images):
    data =[]
    gt_dist = []
    for bias in range(180):
        spc = space_carving_2_masks(data_path,gt_mode=True)
        dist = 180//n_images
        for i in range(0,180,dist):
            spc.carve(calculate_position(i,bias))
        gt_sim = spc.gt_compare_solid(spc.sc.values())
        gt_dist.append(gt_sim)
        h = np.histogram(spc.sc.values(), bins=3)[0]
        data.append(h)
        print(bias,h,gt_sim)
    return data,gt_dist

def test_random(data_path,n_images):
    data =[]
    gt_dist = []
    for n in range(180):
        spc = space_carving_2_masks(data_path,gt_mode=True)
        pos = np.random.randint(180, size=n_images)
        for i in pos:
            spc.carve(i)
        gt_sim = spc.gt_compare_solid(spc.sc.values())
        gt_dist.append(gt_sim)
        h = np.histogram(spc.sc.values(), bins=3)[0]
        data.append(h)
        print(h,gt_sim)
    return data,gt_dist

def test_all(data_path):
    spc = space_carving_2_masks(data_path)
    for i in range(180):#pos:
        spc.carve(i)
        count = np.histogram(spc.sc.values(), bins=3)[0]
        print(i,count)
    return count

In [15]:
data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/000'
#print('count_rand: mean ',np.mean(count_rand,axis=0), ' std ',np.std(count_rand,axis=0) )
#print('count_uni', np.mean(count_uni,axis=0), ' std ',np.std(count_uni,axis=0) )
#print('count_all',count_all)

count_rand,dist_rand = test_random(data_path,10)
count_uni, dist_uni = test_uniform(data_path,10)
count_all = test_all(data_path)

print('count_rand',np.mean(count_rand,axis=0),np.mean(dist_rand))
print('count_uni', np.mean(count_uni,axis=0),np.mean(dist_uni))
print('count_all',count_all)

[666107  11763   4306] 0.5512999071494893
[665959  11901   4316] 0.5500231588698472
[663973  12553   5650] 0.420205237084218
[664160  12725   5291] 0.44870583789911206
[665065  12463   4648] 0.510752688172043
[664396  12231   5549] 0.42785083768690324
[655887  17778   8511] 0.25756525756525755
[662941  13064   6171] 0.38473999676008425
[666226  11893   4057] 0.585119487558512
[665922  12036   4218] 0.5627962085308057
[661694  14350   6132] 0.3871861754157157
[662340  12568   7268] 0.3255909840571743
[665136  12496   4544] 0.5224373075230972
[663919  12859   5398] 0.4398148148148148
[661813  13859   6504] 0.36504764832462344
[664629  12443   5104] 0.4651390520955738
[663708  12532   5936] 0.40020212228398183
[649275  22375  10526] 0.22558890577507598
[664667  12452   5057] 0.4694603676615932
[664063  13728   4385] 0.541372236152268
[663655  12745   5776] 0.41104188300449984
[660659  13135   8382] 0.2832776717557252
[661047  14535   6594] 0.36027293404094013
[666027  11908   4241] 0.5597

14 [666981  11477   3718] 0.6384408602150538
15 [667052  11472   3652] 0.6499726327312534
16 [667053  11466   3657] 0.6490844493030883
17 [667098  11448   3630] 0.6539096916299559
18 [667114  11457   3605] 0.6584419184918214
19 [667052  11464   3660] 0.6485527034407428
20 [666801  11457   3918] 0.6058673469387755
21 [666535  11460   4181] 0.5677743246473823
22 [666743  11456   3977] 0.5968836391053028
23 [666792  11457   3927] 0.6044795113260372
24 [666722  11452   4002] 0.5935548338745941
25 [666809  11447   3920] 0.6059678653404744
26 [666790  11437   3949] 0.6015189873417721
27 [666869  11460   3847] 0.6170433878929592
28 [666906  11455   3815] 0.6222164003143831
29 [666858  11466   3852] 0.6162428645563052
30 [666887  11470   3819] 0.6215650353310652
31 [666933  11480   3763] 0.6308100929614874
32 [666981  11477   3718] 0.6384408602150538
33 [667052  11472   3652] 0.6499726327312534
34 [667053  11466   3657] 0.6490844493030883
35 [667098  11448   3630] 0.6539096916299559
36 [667114

28 [520902 154312   6962]
29 [522033 153309   6834]
30 [523098 152341   6737]
31 [524169 151349   6658]
32 [525225 150348   6603]
33 [526227 149384   6565]
34 [527270 148399   6507]
35 [528329 147400   6447]
36 [529361 146419   6396]
37 [530400 145446   6330]
38 [531484 144447   6245]
39 [532511 143442   6223]
40 [533476 142457   6243]
41 [534470 141459   6247]
42 [535491 140450   6235]
43 [536592 139432   6152]
44 [537723 138356   6097]
45 [538753 137340   6083]
46 [539896 136250   6030]
47 [541048 135133   5995]
48 [542163 134049   5964]
49 [543488 132872   5816]
50 [544634 131745   5797]
51 [545837 130511   5828]
52 [546998 129317   5861]
53 [548208 128075   5893]
54 [549478 126790   5908]
55 [550815 125514   5847]
56 [552218 124144   5814]
57 [553609 122782   5785]
58 [555121 121286   5769]
59 [556656 119753   5767]
60 [558211 118230   5735]
61 [559866 116569   5741]
62 [561581 114835   5760]
63 [563344 113032   5800]
64 [565246 111039   5891]
65 [567171 109078   5927]
66 [569293 1